# Ensemble + Fractional Translation (v2)

Key fix: Validate that solutions don't have overlaps before accepting them.

In [1]:
import sys
import os
os.chdir('/home/code/experiments/007_ensemble_fractional')
sys.path.insert(0, '/home/code')

import numpy as np
import pandas as pd
import json
import time
import glob
from numba import njit

from code.tree_geometry import TX, TY, calculate_score, calculate_bbox
from code.overlap_check import has_overlap
from code.utils import parse_submission, save_submission

print("Imports done")

Imports done


In [2]:
def strip(a):
    """Remove 's' prefix from values."""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_and_validate_csv(fp):
    """Load a submission CSV and validate each N group."""
    try:
        df = pd.read_csv(fp)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            return None
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        return df
    except:
        return None

print("Utility functions defined")

Utility functions defined


In [3]:
# Collect all CSV files from snapshots
snapshot_dir = '/home/nonroot/snapshots/santa-2025'
csv_files = []
for root, dirs, files in os.walk(snapshot_dir):
    for f in files:
        if f.endswith('.csv'):
            csv_files.append(os.path.join(root, f))

# Also add our baseline (which we know is valid)
csv_files.append('/home/code/experiments/001_valid_baseline/submission.csv')

print(f"Found {len(csv_files)} CSV files to ensemble")

Found 3530 CSV files to ensemble


In [4]:
# Build ensemble - best per-N from all sources WITH VALIDATION
best_per_n = {n: {'score': 1e300, 'config': None, 'src': None} for n in range(1, 201)}

print("Processing CSV files with overlap validation...")
processed = 0
valid_found = 0

for fp in csv_files:
    df = load_and_validate_csv(fp)
    if df is None:
        continue
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        
        # Create config as list of tuples
        config = [(xs[i], ys[i], ds[i]) for i in range(len(xs))]
        
        # Validate - no overlaps
        if has_overlap(config):
            continue
        
        # Calculate score
        sc = calculate_score(config)
        valid_found += 1
        
        if sc < best_per_n[n]['score']:
            best_per_n[n]['score'] = sc
            best_per_n[n]['config'] = config
            best_per_n[n]['src'] = fp.split('/')[-2]
    
    processed += 1
    if processed % 500 == 0:
        print(f"  Processed {processed} files, found {valid_found} valid configs...")

print(f"\nProcessed {processed} files total")
print(f"Found {valid_found} valid configurations")

# Calculate ensemble score
ensemble_score = sum(best_per_n[n]['score'] for n in range(1, 201))
print(f"\nEnsemble score: {ensemble_score:.6f}")

Processing CSV files with overlap validation...


  Processed 500 files, found 83999 valid configs...


  Processed 1000 files, found 170171 valid configs...


  Processed 1500 files, found 255997 valid configs...


  Processed 2000 files, found 340166 valid configs...


  Processed 2500 files, found 428669 valid configs...


  Processed 3000 files, found 513842 valid configs...


  Processed 3500 files, found 592189 valid configs...



Processed 3519 files total
Found 595791 valid configurations

Ensemble score: 70.265730


In [5]:
# Check sample scores
print("\nSample per-N scores:")
for n in [1, 2, 5, 10, 20, 50, 100, 150, 200]:
    entry = best_per_n[n]
    print(f"  N={n}: {entry['score']:.6f} (from {entry['src']})")


Sample per-N scores:
  N=1: 0.661250 (from code)
  N=2: 0.450779 (from preoptimized)
  N=5: 0.416850 (from code)
  N=10: 0.376630 (from 013_long_sa)
  N=20: 0.376057 (from 013_long_sa)
  N=50: 0.360753 (from code)
  N=100: 0.343394 (from code)
  N=150: 0.337064 (from code)
  N=200: 0.337549 (from code)


In [6]:
# Load baseline for comparison
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')
baseline_configs = parse_submission(baseline_df)

baseline_scores = {}
for n in range(1, 201):
    baseline_scores[n] = calculate_score(baseline_configs[n])

baseline_total = sum(baseline_scores.values())
print(f"Baseline total: {baseline_total:.6f}")
print(f"Ensemble total: {ensemble_score:.6f}")
print(f"Improvement: {baseline_total - ensemble_score:.9f}")

Baseline total: 70.615102
Ensemble total: 70.265730
Improvement: 0.349372300


In [7]:
# Find improvements over baseline
improvements = []
for n in range(1, 201):
    ens_score = best_per_n[n]['score']
    base_score = baseline_scores[n]
    if ens_score < base_score - 1e-9:
        diff = base_score - ens_score
        improvements.append((n, diff, best_per_n[n]['src']))

print(f"\nFound {len(improvements)} improvements over baseline:")
for n, diff, src in improvements[:20]:
    print(f"  N={n}: improved by {diff:.9f} (from {src})")


Found 43 improvements over baseline:
  N=15: improved by 0.000007550 (from 016_multi_seed_sa)
  N=24: improved by 0.348368973 (from experiments)
  N=35: improved by 0.000015456 (from 016_multi_seed_sa)
  N=36: improved by 0.000333559 (from 016_multi_seed_sa)
  N=57: improved by 0.000027633 (from 016_multi_seed_sa)
  N=58: improved by 0.000001355 (from code)
  N=59: improved by 0.000000974 (from 016_multi_seed_sa)
  N=61: improved by 0.000000330 (from code)
  N=63: improved by 0.000000942 (from code)
  N=64: improved by 0.000001317 (from code)
  N=65: improved by 0.000326870 (from 016_multi_seed_sa)
  N=66: improved by 0.000002632 (from 016_multi_seed_sa)
  N=73: improved by 0.000000750 (from solutions)
  N=76: improved by 0.000000313 (from 016_multi_seed_sa)
  N=78: improved by 0.000002320 (from code)
  N=81: improved by 0.000001643 (from code)
  N=87: improved by 0.000002791 (from code)
  N=88: improved by 0.000007482 (from code)
  N=91: improved by 0.000000705 (from solutions)
  N=9

In [8]:
# Create final configs - use ensemble where better, baseline otherwise
final_configs = {}
for n in range(1, 201):
    if best_per_n[n]['score'] < baseline_scores[n] - 1e-9:
        final_configs[n] = best_per_n[n]['config']
    else:
        final_configs[n] = list(baseline_configs[n])

# Calculate final score
final_score = sum(calculate_score(final_configs[n]) for n in range(1, 201))
print(f"\nFinal score: {final_score:.6f}")
print(f"Baseline: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - final_score:.9f}")


Final score: 70.265730
Baseline: 70.615102
Improvement: 0.349372288


In [9]:
# Save submission
save_submission(final_configs, 'submission.csv')
print("Saved submission.csv")

# Copy to submission folder
import shutil
shutil.copy('submission.csv', '/home/submission/submission.csv')
print("Copied to /home/submission/")

Saved submission.csv
Copied to /home/submission/


In [10]:
# Save metrics
metrics = {
    'cv_score': final_score,
    'baseline_score': baseline_total,
    'ensemble_score': ensemble_score,
    'improvement': baseline_total - final_score,
    'n_improvements': len(improvements),
    'improvements': [(n, float(d), src) for n, d, src in improvements[:20]],
    'notes': 'Ensemble from all snapshots with overlap validation'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {final_score:.6f}")


Final CV Score: 70.265730
